In [29]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.getcwd() + '/Modules/')
from utc_converter import utc_converter
from metar_decoder import metar_decoder
pd.set_option('display.max_columns', None)

#### Data preprocessing
---

In [30]:
# load data frame
fw = pd.read_csv("..//Datasets/NEW/FW_with_airports.csv")

In [31]:
# add time with UTC format 
fw = utc_converter(fw)

In [32]:
# convert date to datetime
fw.date = pd.to_datetime(fw.date, format='%Y-%m-%d')

In [33]:
# split ICAO_code
fw['ICAO_origin'] = fw.origin.str.rsplit("(", n = 1, expand = True)[1].str.split(")", n = 1, expand = True)[0]
fw['ICAO_destination'] = fw.destination.str.rsplit("(", n = 1, expand = True)[1].str.split(")", n = 1, expand = True)[0]

In [34]:
# Split dual ICAO and take the last one
fw.loc[fw.ICAO_origin.str.contains(' / ', na=False), 'ICAO_origin'] = fw.loc[fw.ICAO_origin.str.contains(' / ', na=False), 'ICAO_origin']\
                                                                        .str.split(" / ", expand=True, n=1)[1]

fw.loc[fw.ICAO_destination.str.contains(' / ', na=False), 'ICAO_destination'] = fw.loc[fw.ICAO_destination.str.contains(' / ', na=False), 'ICAO_destination']\
                                                                                  .str.split(" / ", expand=True, n=1)[1]

In [35]:
# replace null ICAO code by airport code
fw.loc[fw.ICAO_origin.isna(), 'ICAO_origin'] = fw.loc[fw.ICAO_origin.isna(), 'origin_code'] 
fw.loc[fw.ICAO_destination.isna(), 'ICAO_destination'] = fw.loc[fw.ICAO_destination.isna(), 'destination_code']

In [36]:
# Transform aita code to icao
fw.loc[~fw.ICAO_origin.str.startswith('K', na=False), 'ICAO_origin'] = 'K' + fw.loc[~fw.ICAO_origin.str.startswith('K', na=False), 'ICAO_origin']
fw.loc[~fw.ICAO_destination.str.startswith('K', na=False), 'ICAO_destination'] = 'K' + fw.loc[~fw.ICAO_destination.str.startswith('K', na=False), 'ICAO_destination']

In [37]:
# extract hour from dep_UTC_time and assign a new variable
fw['dep_UTC_hour'] = pd.to_datetime(fw.dep_UTC_time.astype('str'), format='%H:%M:%S', errors='coerce').dt.hour.astype('str')
fw['arr_UTC_hour'] = pd.to_datetime(fw.arr_UTC_time.astype('str'), format='%H:%M:%S', errors='coerce').dt.hour.astype('str')

In [38]:
# sort by 'tail_number', 'date', 'dep_UTC_hour'
fw = fw.sort_values(['tail_number', 'date', 'dep_UTC_hour']).reset_index(drop=True)

In [39]:
# inspect data
fw

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,airport_origin,city_origin,origin_state,country_origin,airport_destination,city_destination,destination_state,country_destination,departure_shift,arrival_shift,ICAO_origin,ICAO_destination,dep_UTC_hour,arr_UTC_hour
0,N100KB,2020-11-29,BE9L,KSTP,St Paul Holman Fld (KSTP),KMOT,Minot Intl (KMOT),07:46PM CST,01:46:00,09:53PM CST,03:53:00,2:07,453.428407,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",44.934502,-93.059998,48.259399,-101.279999,St Paul Downtown Holman Field,St Paul,MN,US,Minot International Airport,Minot,ND,US,night,night,KSTP,KMOT,1.0,3.0
1,N100KB,2020-12-07,BE9L,KMOT,Minot Intl (KMOT),KFSD,Joe Foss Fld (KFSD),09:54PM CST,03:54:00,11:24PM CST,05:24:00,1:29,389.979210,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,43.582001,-96.741898,Minot International Airport,Minot,ND,US,Joe Foss Field Airport,Sioux Falls,SD,US,night,night,KMOT,KFSD,3.0,5.0
2,N100KB,2020-12-08,BE9L,KRST,Rochester Intl (KRST),KMOT,Minot Intl (KMOT),06:28PM CST,00:28:00,09:26PM CST,03:26:00,2:57,517.521438,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",43.908298,-92.500000,48.259399,-101.279999,Rochester International Airport,Rochester,MN,US,Minot International Airport,Minot,ND,US,day,night,KRST,KMOT,0.0,3.0
3,N100KB,2020-12-08,BE9L,KMOT,Minot Intl (KMOT),KDVL,Devils Lake Rgnl (KDVL),05:57AM CST,11:57:00,06:25AM CST,12:25:00,0:27,110.009068,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.114201,-98.908798,Minot International Airport,Minot,ND,US,Devils Lake Regional Airport,Devils Lake,ND,US,night,night,KMOT,KDVL,11.0,12.0
4,N100KB,2020-12-08,BE9L,KDVL,Devils Lake Rgnl (KDVL),KMOT,Minot Intl (KMOT),06:49AM CST,12:49:00,07:21AM CST,13:21:00,0:32,110.009068,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.114201,-98.908798,48.259399,-101.279999,Devils Lake Regional Airport,Devils Lake,ND,US,Minot International Airport,Minot,ND,US,night,day,KDVL,KMOT,12.0,13.0


#### METAR data merging for Origin Location
---

In [40]:
# scrape METAR data for Origin
from metar_scraper_origin import metar_scraper_origin
metar_scraper_origin(fw)

KeyboardInterrupt: 

In [15]:
# load data frame
metar_origin = pd.read_csv("..//Datasets/NEW/METAR/origin/FW_METAR_origin.csv")

In [16]:
# split date and time. Extract only hours  
metar_origin['hour'] = pd.to_datetime(metar_origin.date, format='%d/%m/%Y %H:%M').dt.hour.astype('str') + '.0'
metar_origin['date'] = pd.to_datetime(metar_origin.date, format='%d/%m/%Y %H:%M').dt.date.astype('str')

In [17]:
# convert date to string
fw.date = fw.date.astype('str')

In [18]:
# sort by 'tail_number', 'date', 'hour' and drop duplicates
metar_origin = metar_origin.sort_values(['tail_number', 'date', 'hour']).drop_duplicates(subset=['tail_number', 'ICAO_origin', 'date', 'hour'])\
                           .reset_index(drop=True)

In [19]:
# inspect data
metar_origin.head()

,tail_number,ICAO_origin,date,METAR,hour
0,N100KB,KSTP,2020-11-29,METAR KSTP 290153Z AUTO 13006KT 10SM CLR 01/M0...,1.0
1,N100KB,KSTP,2020-11-29,METAR KSTP 290253Z AUTO 00000KT 10SM CLR 02/M0...,2.0
2,N100KB,KMOT,2020-12-06,METAR KMOT 061854Z 27004KT 10SM CLR 09/M04 A30...,18.0
3,N100KB,KMOT,2020-12-06,METAR KMOT 061954Z 29006KT 10SM CLR 11/M04 A30...,19.0
4,N100KB,KMOT,2020-12-07,METAR KMOT 070354Z 24012KT 10SM CLR 02/M04 A30...,3.0


In [20]:
# merge by 'tail_number', 'date', 'ICAO_origin', 'dep_UTC_hour'
fw = fw.merge(metar_origin.add_suffix('_origin'), how='left', left_on=['tail_number', 'date', 'ICAO_origin', 'dep_UTC_hour'],
        right_on=['tail_number_origin', 'date_origin', 'ICAO_origin_origin', 'hour_origin'])\
        .drop(['ICAO_origin', 'dep_UTC_hour','tail_number_origin', 'ICAO_origin_origin', 'date_origin', 'hour_origin'], axis=1)

In [21]:
# inspect merging
fw.head()

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,airport_origin,city_origin,origin_state,country_origin,airport_destination,city_destination,destination_state,country_destination,departure_shift,arrival_shift,ICAO_destination,arr_UTC_hour,METAR_origin
0,N100KB,2020-11-29,BE9L,KSTP,St Paul Holman Fld (KSTP),KMOT,Minot Intl (KMOT),07:46PM CST,01:46:00,09:53PM CST,03:53:00,2:07,453.428407,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",44.934502,-93.059998,48.259399,-101.279999,St Paul Downtown Holman Field,St Paul,MN,US,Minot International Airport,Minot,ND,US,night,night,KMOT,3.0,METAR KSTP 290153Z AUTO 13006KT 10SM CLR 01/M0...
1,N100KB,2020-12-07,BE9L,KMOT,Minot Intl (KMOT),KFSD,Joe Foss Fld (KFSD),09:54PM CST,03:54:00,11:24PM CST,05:24:00,1:29,389.979210,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,43.582001,-96.741898,Minot International Airport,Minot,ND,US,Joe Foss Field Airport,Sioux Falls,SD,US,night,night,KFSD,5.0,METAR KMOT 070354Z 24012KT 10SM CLR 02/M04 A30...
2,N100KB,2020-12-08,BE9L,KRST,Rochester Intl (KRST),KMOT,Minot Intl (KMOT),06:28PM CST,00:28:00,09:26PM CST,03:26:00,2:57,517.521438,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",43.908298,-92.500000,48.259399,-101.279999,Rochester International Airport,Rochester,MN,US,Minot International Airport,Minot,ND,US,day,night,KMOT,3.0,METAR KRST 080054Z 21005KT 6SM BR OVC007 M02/M...
3,N100KB,2020-12-08,BE9L,KMOT,Minot Intl (KMOT),KDVL,Devils Lake Rgnl (KDVL),05:57AM CST,11:57:00,06:25AM CST,12:25:00,0:27,110.009068,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.114201,-98.908798,Minot International Airport,Minot,ND,US,Devils Lake Regional Airport,Devils Lake,ND,US,night,night,KDVL,12.0,METAR KMOT 081154Z AUTO 23008KT 10SM CLR 03/M0...
4,N100KB,2020-12-08,BE9L,KDVL,Devils Lake Rgnl (KDVL),KMOT,Minot Intl (KMOT),06:49AM CST,12:49:00,07:21AM CST,13:21:00,0:32,110.009068,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.114201,-98.908798,48.259399,-101.279999,Devils Lake Regional Airport,Devils Lake,ND,US,Minot International Airport,Minot,ND,US,night,day,KMOT,13.0,METAR KDVL 081256Z AUTO 27008KT 10SM CLR 01/M0...


In [22]:
# check for null values
fw.METAR_origin.isna().sum()

2810

In [25]:
fw.loc[fw.METAR_origin.isna()]

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,airport_origin,city_origin,origin_state,country_origin,airport_destination,city_destination,destination_state,country_destination,departure_shift,arrival_shift,ICAO_destination,arr_UTC_hour,METAR_origin
9,N100KB,2020-12-08,BE9L,KFSD,Joe Foss Fld (KFSD),KMOT,Minot Intl (KMOT),12:38AM CST,06:38:00,02:40AM CST,08:40:00,2:02,389.979210,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",43.582001,-96.741898,48.259399,-101.279999,Joe Foss Field Airport,Sioux Falls,SD,US,Minot International Airport,Minot,ND,US,night,night,KMOT,8.0,NaN
60,N100KB,2021-02-07,BE9L,CJJ4,"Deloraine, MB",KBIS,Bismarck Muni (KBIS),09:47PM CST,03:47:00,10:18PM CST,04:18:00,0:31,140.761379,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.795780,-100.387790,46.772701,-100.746002,Deloraine Airport,Deloraine,MB,CA,Bismarck Municipal Airport,Bismarck,ND,US,night,night,KBIS,4.0,NaN
332,N114HB,2020-11-30,BE20,KE06,Lea County-Zip Franklin Meml (E06),KABQ,Albuquerque Intl Sunport (KABQ),08:29AM MST,15:29:00,09:22AM MST,16:22:00,0:52,233.265242,"EAGLEMED LLCWICHITA, KS, US",32.953899,-103.408997,35.040199,-106.609001,Lea County-Zip Franklin Memorial Airport,Lovington,NM,US,Albuquerque International Sunport,Albuquerque,NM,US,day,day,KABQ,16.0,NaN
339,N114HB,2020-12-01,BE20,KLLN,Levelland Muni (KLLN),KT82,Gillespie County (T82),10:35AM CST,16:35:00,11:32AM CST,17:32:00,0:56,305.601256,"EAGLEMED LLCWICHITA, KS, US",33.552502,-102.372002,30.243200,-98.909203,Levelland Municipal Airport,Levelland,TX,US,Gillespie County Airport,Fredericksburg,TX,US,day,day,KT82,17.0,NaN
342,N114HB,2020-12-01,BE20,KE06,Lea County-Zip Franklin Meml (E06),KLRU,Las Cruces Intl (KLRU),08:54PM MST,03:54:00,09:51PM MST,04:51:00,0:57,209.906508,"EAGLEMED LLCWICHITA, KS, US",32.953899,-103.408997,32.289398,-106.921997,Lea County-Zip Franklin Memorial Airport,Lovington,NM,US,Las Cruces International Airport,Las Cruces,NM,US,night,night,KLRU,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21914,N986AC,2021-03-02,PC12,K0D8,Gettysburg Muni (0D8),KFSD,Joe Foss Fld (KFSD),11:53AM CST,17:53:00,12:36PM CST,18:36:00,0:42,186.444669,"AVERA MCKENNANSIOUX FALLS, SD, US(Corporation)",44.986698,-99.952797,43.582001,-96.741898,Gettysburg Municipal Airport,Gettysburg,SD,US,Joe Foss Field Airport,Sioux Falls,SD,US,day,day,KFSD,18.0,NaN
21925,N986AC,2021-03-05,PC12,KSUO,Rosebud Sioux Tribal (KSUO),KFSD,Joe Foss Fld (KFSD),08:05PM CST,02:05:00,08:49PM CST,02:49:00,0:44,208.379078,"AVERA MCKENNANSIOUX FALLS, SD, US(Corporation)",43.258500,-100.859520,43.582001,-96.741898,Rosebud Sioux Tribal Airport,Rosebud,SD,US,Joe Foss Field Airport,Sioux Falls,SD,US,night,night,KFSD,2.0,NaN
21931,N986AC,2021-03-06,PC12,KMKA,Miller Muni (KMKA),KFSD,Joe Foss Fld (KFSD),03:23PM CST,21:23:00,03:54PM CST,21:54:00,0:31,128.126240,"AVERA MCKENNANSIOUX FALLS, SD, US(Corporation)",44.525200,-98.958099,43.582001,-96.741898,Miller Municipal Airport,Miller,SD,US,Joe Foss Field Airport,Sioux Falls,SD,US,day,day,KFSD,21.0,NaN
21936,N986AC,2021-03-07,PC12,K84D,Cheyenne Eagle Butte (84D),KRAP,Rapid City Rgnl (KRAP),06:13AM MST,13:13:00,06:43AM MST,13:43:00,0:30,110.293951,"AVERA MCKENNANSIOUX FALLS, SD, US(Corporation)",44.984402,-101.251000,44.045300,-103.056999,Cheyenne Eagle Butte Airport,Eagle Butte,SD,US,Rapid City Regional Airport,Rapid City,SD,US,night,night,KRAP,13.0,NaN


#### METAR data merging for Destination Location
---

In [27]:
# scrape METAR data for Origin
from metar_scraper_destination import metar_scraper_destination
metar_scraper_destination(fw)

KeyboardInterrupt: 

In [26]:
# load data frame
metar_destination = pd.read_csv("../Datasets/METAR/FW/destination/FW_METAR_destination.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/METAR/FW/destination/FW_METAR_destination.csv'

In [70]:
# split date and time. Extract only hours  
metar_destination['hour'] = pd.to_datetime(metar_destination.date, format='%d/%m/%Y %H:%M').dt.hour.astype('str') + '.0'
metar_destination['date'] = pd.to_datetime(metar_destination.date, format='%d/%m/%Y %H:%M').dt.date.astype('str')

In [71]:
# sort by 'tail_number', 'date', 'hour' and drop duplicates
metar_destination = metar_destination.sort_values(['tail_number', 'date', 'hour']).drop_duplicates(subset=['tail_number', 'ICAO_destination', 'date', 'hour'])\
                           .reset_index(drop=True)

In [72]:
# inspect data
metar_destination.head()

,tail_number,ICAO_destination,date,METAR,hour
0,N100KB,KMOT,2020-11-29,METAR KMOT 290354Z 29023G34KT 10SM SCT031 SCT0...,3.0
1,N100KB,KMOT,2020-11-29,METAR KMOT 290454Z AUTO 30024G36KT 10SM FEW024...,4.0
2,N100KB,KMOT,2020-12-06,METAR KMOT 061854Z 27004KT 10SM CLR 09/M04 A30...,18.0
3,N100KB,KMOT,2020-12-06,METAR KMOT 061954Z 29006KT 10SM CLR 11/M04 A30...,19.0
4,N100KB,KFSD,2020-12-07,METAR KFSD 070556Z 00000KT 10SM CLR M04/M06 A3...,5.0


In [73]:
# merge by 'tail_number', 'date', 'ICAO_origin', 'arr_UTC_hour'
fw = fw.merge(metar_destination.add_suffix('_destination'), how='left', left_on=['tail_number', 'date', 'ICAO_destination', 'arr_UTC_hour'],
        right_on=['tail_number_destination', 'date_destination', 'ICAO_destination_destination', 'hour_destination'])\
        .drop(['ICAO_destination', 'arr_UTC_hour','tail_number_destination', 'ICAO_destination_destination', 
               'date_destination', 'hour_destination'], axis=1)

In [74]:
# inspect data
fw.head()

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,airport_origin,city_origin,origin_state,country_origin,origin_Latitude,origin_Longitude,airport_destination,city_destination,destination_state,country_destination,destination_Latitude,destination_Longitude,Owner,METAR_origin,METAR_destination
0,N100KB,2020-11-29,BE9L,KSTP,St Paul Holman Fld (KSTP),KMOT,Minot Intl (KMOT),07:46PM CST,01:46:00,09:53PM CST,03:53:00,2:07,453.428407,St Paul Downtown Holman Field,St Paul - MN,MN,US,44.934502,-93.059998,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KSTP 290153Z AUTO 13006KT 10SM CLR 01/M0...,METAR KMOT 290354Z 29023G34KT 10SM SCT031 SCT0...
1,N100KB,2020-12-06,BE9L,KMOT,Minot Intl (KMOT),KMOT,Minot Intl (KMOT),12:07PM CST,18:07:00,12:21PM CST,18:21:00,0:13,0.000000,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KMOT 061854Z 27004KT 10SM CLR 09/M04 A30...,METAR KMOT 061854Z 27004KT 10SM CLR 09/M04 A30...
2,N100KB,2020-12-07,BE9L,KMOT,Minot Intl (KMOT),KFSD,Joe Foss Fld (KFSD),09:54PM CST,03:54:00,11:24PM CST,05:24:00,1:29,389.979210,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,Joe Foss Field Airport,Sioux Falls - SD,SD,US,43.582001,-96.741898,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KMOT 070354Z 24012KT 10SM CLR 02/M04 A30...,METAR KFSD 070556Z 00000KT 10SM CLR M04/M06 A3...
3,N100KB,2020-12-08,BE9L,KRST,Rochester Intl (KRST),KMOT,Minot Intl (KMOT),06:28PM CST,00:28:00,09:26PM CST,03:26:00,2:57,517.521438,Rochester International Airport,Rochester - MN,MN,US,43.908298,-92.500000,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KRST 080054Z 21005KT 6SM BR OVC007 M02/M...,METAR KMOT 080354Z 26015KT 10SM CLR 06/M03 A29...
4,N100KB,2020-12-08,BE9L,KMOT,Minot Intl (KMOT),KDVL,Devils Lake Rgnl (KDVL),05:57AM CST,11:57:00,06:25AM CST,12:25:00,0:27,110.009068,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,Devils Lake Regional Airport,Devils Lake - ND,ND,US,48.114201,-98.908798,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KMOT 081154Z AUTO 23008KT 10SM CLR 03/M0...,METAR KDVL 081256Z AUTO 27008KT 10SM CLR 01/M0...


In [75]:
# check for null values
fw.METAR_destination.isna().sum()

3444

#### METAR Decoding
---

In [76]:
# decode metar data
fw = metar_decoder(fw)

In [77]:
fw = fw.loc[:,:'METAR_origin'].join([fw.loc[:,'METAR_time_origin':'remarks_origin'], 
                                fw.loc[:,'METAR_destination'], fw.loc[:,'METAR_time_destination':'remarks_destination']])

In [78]:
# inspect data
fw.head()

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,airport_origin,city_origin,origin_state,country_origin,origin_Latitude,origin_Longitude,airport_destination,city_destination,destination_state,country_destination,destination_Latitude,destination_Longitude,Owner,METAR_origin,METAR_time_origin,report_type_origin,temperature_origin,dew_point_origin,wind_origin,wind_peak_origin,visibility_origin,pressure_origin,press_sea_level_origin,sky_origin,remarks_origin,METAR_destination,METAR_time_destination,report_type_destination,temperature_destination,dew_point_destination,wind_destination,wind_peak_destination,visibility_destination,pressure_destination,press_sea_level_destination,sky_destination,remarks_destination
0,N100KB,2020-11-29,BE9L,KSTP,St Paul Holman Fld (KSTP),KMOT,Minot Intl (KMOT),07:46PM CST,01:46:00,09:53PM CST,03:53:00,2:07,453.428407,St Paul Downtown Holman Field,St Paul - MN,MN,US,44.934502,-93.059998,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KSTP 290153Z AUTO 13006KT 10SM CLR 01/M0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METAR KMOT 290354Z 29023G34KT 10SM SCT031 SCT0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N100KB,2020-12-06,BE9L,KMOT,Minot Intl (KMOT),KMOT,Minot Intl (KMOT),12:07PM CST,18:07:00,12:21PM CST,18:21:00,0:13,0.000000,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KMOT 061854Z 27004KT 10SM CLR 09/M04 A30...,18:54:00,"routine report, cycle 19 (automatic report)",48.9 F,25.0 F,W at 4 knots,missing,10 miles,1018.0 mb,1019.4 mb,clear,Automated station (type 2),METAR KMOT 061854Z 27004KT 10SM CLR 09/M04 A30...,18:54:00,"routine report, cycle 19 (automatic report)",48.9 F,25.0 F,W at 4 knots,missing,10 miles,1018.0 mb,1019.4 mb,clear,Automated station (type 2)
2,N100KB,2020-12-07,BE9L,KMOT,Minot Intl (KMOT),KFSD,Joe Foss Fld (KFSD),09:54PM CST,03:54:00,11:24PM CST,05:24:00,1:29,389.979210,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,Joe Foss Field Airport,Sioux Falls - SD,SD,US,43.582001,-96.741898,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KMOT 070354Z 24012KT 10SM CLR 02/M04 A30...,03:54:00,"routine report, cycle 4 (automatic report)",35.1 F,24.1 F,WSW at 12 knots,missing,10 miles,1016.6 mb,1018.5 mb,clear,Automated station (type 2),METAR KFSD 070556Z 00000KT 10SM CLR M04/M06 A3...,05:56:00,"routine report, cycle 6 (automatic report)",25.0 F,21.9 F,calm,missing,10 miles,1017.3 mb,1019.0 mb,clear,Automated station (type 2); 3-hr pressure chan...
3,N100KB,2020-12-08,BE9L,KRST,Rochester Intl (KRST),KMOT,Minot Intl (KMOT),06:28PM CST,00:28:00,09:26PM CST,03:26:00,2:57,517.521438,Rochester International Airport,Rochester - MN,MN,US,43.908298,-92.500000,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KRST 080054Z 21005KT 6SM BR OVC007 M02/M...,00:54:00,"routine report, cycle 1 (automatic report)",28.9 F,26.1 F,SSW at 5 knots,missing,6 miles,1018.0 mb,1019.5 mb,overcast at 700 feet,Automated station (type 2),METAR KMOT 080354Z 26015KT 10SM CLR 06/M03 A29...,03:54:00,"routine report, cycle 4 (automatic report)",42.1 F,26.1 F,W at 15 knots,missing,10 miles,1014.2 mb,1015.9 mb,clear,Automated station (type 2)
4,N100KB,2020-12-08,BE9L,KMOT,Minot Intl (KMOT),KDVL,Devils Lake Rgnl (KDVL),05:57AM CST,11:57:00,06:25AM CST,12:25:00,0:27,110.009068,Minot International Airport,Minot - ND,ND,US,48.259399,-101.279999,Devils Lake Regional Airport,Devils Lake - ND,ND,US,48.114201,-98.908798,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",METAR KMOT 081154Z AUTO 23008KT 10SM CLR 03/M0...,11:54:00,"routine report, cycle 12 (automatic report)",37.9 F,27.0 F,SW at 8 knots,missing,10 miles,10

In [79]:
fw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23160 entries, 0 to 23159
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   tail_number                  23160 non-null  object 
 1   date                         23160 non-null  object 
 2   aircraft                     22966 non-null  object 
 3   origin_code                  23142 non-null  object 
 4   origin                       23160 non-null  object 
 5   destination_code             23121 non-null  object 
 6   destination                  23159 non-null  object 
 7   departure                    23160 non-null  object 
 8   dep_UTC_time                 23148 non-null  object 
 9   arrival                      22730 non-null  object 
 10  arr_UTC_time                 22717 non-null  object 
 11  duration                     22680 non-null  object 
 12  distance_mi                  23151 non-null  float64
 13  airport_origin  

In [81]:
fw.isna().sum()

tail_number                       0
date                              0
aircraft                        194
origin_code                      18
origin                            0
destination_code                 39
destination                       1
departure                         0
dep_UTC_time                     12
arrival                         430
arr_UTC_time                    443
duration                        480
distance_mi                       9
airport_origin                   18
city_origin                       0
origin_state                     35
country_origin                   18
origin_Latitude                   3
origin_Longitude                  3
airport_destination              39
city_destination                  1
destination_state                37
country_destination              39
destination_Latitude              6
destination_Longitude             6
Owner                             0
METAR_origin                   2964
METAR_time_origin           

In [80]:
# export as csv
fw.to_csv('../Datasets/NEW/fw_with_metar.csv', index=False)